# **Project pRF fitting results to the Freesurfer surfaces**

### Initiation - Variables and paths

In [ ]:
# what subject?
SUBJ=Danny

# where are the volume-based results
pRFVol=/Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI/${SUBJ,}

# where will the result be saved?
OUT=/Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/Results_on_Surface/pRF_Surf_${SUBJ}
mkdir -p $OUT

# where can we find the epi space the results are in?
# This is usually the reference from the `manual-masks' folder that is used for preprocessing and modelfit in NHP-BIDS.
EPI_ROOT=/Users/chris/Documents/MRI_ANALYSIS/NHP-BIDS/manual-masks/sub-${SUBJ,}/func/
EPI=${EPI_ROOT}/sub-${SUBJ,}_ref_func_res-1x1x1.nii.gz
EPI_MASK=${EPI_ROOT}/sub-${SUBJ,}_ref_func_mask_res-1x1x1.nii.gz

# where is the anatomical you'd like to register to?
# $SUBJECTS_DIR should point to the Freesurfer subjects directory. Adjust your bashrc to export this if it doesn't do so already.
T1=${SUBJECTS_DIR}/${SUBJ}/mri/brainmask.mgz
# and the corresponding white matter segmentation?
WM=${SUBJECTS_DIR}/${SUBJ}/mri/wm.mgz
# where's the surface folder for this subject?
FSSURF=${SUBJECTS_DIR}/${SUBJ}/surf

In [ ]:
# copy the surfaces in the outputfolder
cp -r ${FSSURF} ${OUT}/fs_surf

In [ ]:
# Mask the epi that is used as a reference in functional analysis to extract the brain
fslmaths ${EPI} -mas ${EPI_MASK} ${OUT}/epi_brain.nii.gz
EPI_BRAIN=${OUT}/epi_brain.nii.gz

## Flirt registration of epi to T1
The T1 you pick here is the header adjusted one that formed the based of the surface generation in Freesurfer. We do not have to adjust the header of the epi, because flirt will generally be able to take of the scaling. If this fails for some reason you can still do it manually with `3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen <epi>.nii.gz`. Note that we set voxel sizes to 2 mm in the header in this case. The reason is that for the T1 we had 0.5 mm voxels and adjusted the header to 1 mm, a factor of 2. Our pre-processed epi's have 1 mm voxels, so applying the same factor of 2 means the header info should state that the voxels are 2 mm isotropic.

Be aware that, when you first adjust the header and then do flirt, the final registration matrix will not include this scaling and expect header-adjusted input. Any result you want to warp to the surface will thus first have to be adjusted. You don't have to do this is if the flirt is performed on the non-adjusted epi. 

NB1! One thing to carefully check is whether the x-direction of the voxel order matches between the epi and T1 because if it doesn't we will see left/right flips. With some of our older standard epi's this needs correction (later I corrected it in the reference file for pre-processing). There's a script that does that for you called `swap_xdir_voxels.sh`. You can find it in the `bin` folder of the `Process-NHP-MRI` repository (https://github.com/VisionandCognition/Process-NHP-MRI)

NB2! flirt works significantly better if you include the white matter segmentation (`-wmseg`) of the T1 and phase encoding direction (`-pedir`) of the epi. Check the documentation to find how to code this. If you used Chris Klink's standard epi sequence, your `-pedir` is `-2`.

In [ ]:
# get the brain and white matter volumes from freesurfer & convert to nifti
mri_convert ${T1} ${OUT}/brain.nii.gz 
mri_convert ${WM} ${OUT}/wm.nii.gz 
# calculate the registration
flirt -ref ${OUT}/brain.nii.gz -wmseg ${OUT}/wm.nii.gz -in ${EPI_BRAIN} -out ${OUT}/epi2anat.nii.gz -omat ${OUT}/epi2anat.mat -pedir -2
# check whether nonlinear registration improves on this result (could be a mess as well)
# fnirt --ref ${OUT}/brain.nii.gz --in ${EPI_BRAIN} --aff ${OUT}/epi2anat.mat --iout ${OUT}/epi2anat_fnirt.nii.gz --inmask ${EPI_MASK} 

## Create the tkregister matrix
Now we will use the flirt registration matrix to create a registration matrix in freesurfer format. To get there, we can use the freesurfer program `tkregister` that allows manual registration between 2 volumes. We will inititate it with the flirt matrix and check whether registration is good. If not, you can make manual adjustments, but flirt probably does a better job than manual attempts. Within `tkregister` you can check the alignment of the two volumes by clicking `compare`. Save the registration matrix as `reg.fsl.dat`.

In [ ]:
# run the tkregister registration initated with the flirt transform
tkregister2 --mov ${EPI_BRAIN} --targ ${OUT}/brain.nii.gz --fsl ${OUT}/epi2anat.mat --reg ${OUT}/reg.fsl.dat --s ${SUBJ}

In [ ]:
tkReg=${OUT}/reg.fsl.dat 

## Convert volumes to surface representation    
Cycle over models, cv-modes, hrfs, and R2-thresholds (for both hemispheres).

In [ ]:
# values to cycle over
declare -a hrfmod=(mhrf dhrf)
declare -a cv=(cv0 cv1)
declare -a models=(linhrf linhrf_neggain csshrf doghrf)
declare -a TH=(0 1 2 4 5 10)
declare -a HEMI=(lh rh)

We can now convert the statistical volumes to surface representations using the `mri_vol2surf` command. Since this essentially brings a 3d result to 2d there is a choice to be made on how/where to sample. The `--projfrac` tells the command where between the WM/GM border (`--projfract 0`) and the pial surface (`--projfract 1`) to get the data. These fractions can also be negative (going into the WM) or higher than 1 (beyond the pial surface). Alternatively, you can average along the normal between WM/GM border and pial surface using `--projfrac-avg min max stepsize` or the maximum by using `--projfrac-max min max stepsize`.

In [ ]:
# look at the mri_vol2surf documentation for more info
# mri_vol2surf --help

In [14]:
# These (re)definitions allow to do this part independent of previous steps
declare -a subs=(Danny Eddy)
for SUBJ in ${subs[@]}; do
    # SUBJ=Danny

    OUT=/Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/Results_on_Surface/pRF_Surf_${SUBJ}
    pRFVol=/Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI/${SUBJ,}
    tkReg=${OUT}/reg.fsl.dat 

    # pick one setting here, we can cycle later
    for h in ${hrfmod[@]}; do
        for c in ${cv[@]}; do
            for m in ${models[@]}; do
                if [ $m = linhrf_neggain ]; then
                    # Model folder
                    Model_fld=${pRFVol}/linhrf_${c}_${h}_neggain
                    modlabel=linhrf_${c}_${h}_neggain
                else
                    Model_fld=${pRFVol}/${m}_${c}_${h}
                    modlabel=${m}_${c}_${h}
                fi

#                 if [ $c = cv0 ]; then
#                     R2=${Model_fld}/Sess-${modlabel}_R2.nii.gz
#                 else
#                     R2=${Model_fld}/Sess-${modlabel}_maxR2.nii.gz
#                 fi

#                 for xh in ${HEMI[@]}; do
#                     # R2
#                     mri_vol2surf --trgsubject ${SUBJ} --src $R2 --out ${OUT}/${modlabel}/${xh}.R2.w \
#                         --out_type paint --projfrac-max -.1 1.1 .1 --srcreg ${tkReg} --hemi ${xh}
#                 done   

                for t in ${TH[@]}; do
                    # Threshold folder
                    TH_fld=${Model_fld}/TH_${t}
                    TH_OUT=${OUT}/${modlabel}/TH_${t}
                    mkdir -p ${TH_OUT}

                    # Volumes to variables
#                     ANG=${TH_fld}/ANG_th${t}.nii.gz
#                     ECC=${TH_fld}/ECC_th${t}.nii.gz
#                     FWHM=${TH_fld}/FWHM_th${t}.nii.gz
#                     X=${TH_fld}/X_th${t}.nii.gz
#                     Y=${TH_fld}/Y_th${t}.nii.gz
#                     RFS=${TH_fld}/RFS_th${t}.nii.gz
#                     REAL=${TH_fld}/REAL_th${t}.nii.gz
#                     IMAG=${TH_fld}/IMAG_th${t}.nii.gz

                    # Change angles for visualisation
#                     fslmaths $ANG -sub 90 ${TH_fld}/ANGv.nii.gz
#                     fslmaths ${TH_fld}/ANGv.nii.gz -binv -mul 360 -add ${TH_fld}/ANGv.nii.gz ${TH_fld}/ANGv.nii.gz
#                     fslmaths ${TH_fld}/ANGv.nii.gz -mul ${TH_fld}/mask_th${t}.nii.gz -sub ${TH_fld}/invmask_th${t}.nii.gz ${TH_fld}/ANGv.nii.gz
                    ANG=${TH_fld}/ANGv.nii.gz

#                     fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $ANG $ANG
#                     fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $ECC $ECC
#                     fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $FWHM $FWHM
#                     fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $X $X
#                     fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $Y $Y
#                     fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $RFS $RFS
#                     fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $REAL $REAL
#                     fslmaths ${TH_fld}/invmask_th${t}.nii.gz -mul -99 -add $IMAG $IMAG

                    # create surface overlays                
                    for xh in ${HEMI[@]}; do
                        # angles
                        mri_vol2surf --trgsubject ${SUBJ} --src $ANG --out ${TH_OUT}/${xh}.ANGLE.w \
                            --out_type paint --projfrac-max -.1 1.1 .1 --srcreg ${tkReg} --hemi ${xh}
#                         # ecc
#                         mri_vol2surf --trgsubject ${SUBJ} --src $ECC --out ${TH_OUT}/${xh}.ECC.w \
#                             --out_type paint --projfrac-max -.1 1.1 .1 --srcreg ${tkReg} --hemi ${xh}
#                         # fwhm
#                         mri_vol2surf --trgsubject ${SUBJ} --src $FWHM --out ${TH_OUT}/${xh}.FWHM.w \
#                             --out_type paint --projfrac-max -.1 1.1 .1 --srcreg ${tkReg} --hemi ${xh}
#                         # x
#                         mri_vol2surf --trgsubject ${SUBJ} --src $X --out ${TH_OUT}/${xh}.X.w \
#                             --out_type paint --projfrac-max -.1 1.1 .1 --srcreg ${tkReg} --hemi ${xh}
#                         # y
#                         mri_vol2surf --trgsubject ${SUBJ} --src $Y --out ${TH_OUT}/${xh}.Y.w \
#                             --out_type paint --projfrac-max -.1 1.1 .1 --srcreg ${tkReg} --hemi ${xh}
#                         # rfs
#                         mri_vol2surf --trgsubject ${SUBJ} --src $RFS --out ${TH_OUT}/${xh}.RFS.w \
#                             --out_type paint --projfrac-max -.1 1.1 .1 --srcreg ${tkReg} --hemi ${xh}
#                         # real
#                         mri_vol2surf --trgsubject ${SUBJ} --src $REAL --out ${TH_OUT}/${xh}.REAL.w \
#                             --out_type paint --projfrac-max -.1 1.1 .1 --srcreg ${tkReg} --hemi ${xh}
#                         # imag
#                         mri_vol2surf --trgsubject ${SUBJ} --src $IMAG --out ${TH_OUT}/${xh}.IMAG.w \
#                             --out_type paint --projfrac-max -.1 1.1 .1 --srcreg ${tkReg} --hemi ${xh}
                    done
                done
            done
        done
    done
done

INFO: output format is paint
srcvol = /Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI/danny/linhrf_cv0_mhrf/TH_0/ANGv.nii.gz
srcreg = /Users/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/Results_on_Surface/pRF_Surf_Danny/reg.fsl.dat
srcregold = 0
srcwarp unspecified
surf = white
hemi = lh
trgsubject = Danny
surfreg = sphere.reg
ProjFrac = 0.5
thickness = thickness
reshape = 0
interp = nearest
float2int = round
GetProjMax = 1
INFO: float2int code = 0
Done loading volume
INFO: This REGISTER_DAT transform is valid only for volumes between  COR types with c_(r,a,s) = 0.
Input reg is register.dat
-------- original matrix -----------
-0.49320   0.00371   0.00098  -1.01108;
-0.00172  -0.00144   0.48781   0.09871;
-0.00601  -0.49854   0.04358   1.62712;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
Reading surface /Users/chris/Documents/MRI/freesurfer/subjects//Danny/surf/lh.white
Done reading source surface
Reading thickness /

## Visualising the results  
The cells below provide the code for `tkSurfer`, but everything looks a lot more modern in `surfice`

In [ ]:
surfice

In [ ]:
# show in tksurfer
xh=lh # switch hemisphere easily
tksurfer ${SUBJ} ${xh} graymid -patch full.patch.flat -overlay ${OUT}/${xh}.CT3_avg.w -overlay-reg ${tkReg}